In [10]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import yfinance as yf # Import yfinance for fetching ticker data
import requests # Import requests for fetching data from NASDAQ API

# --- Configuration Parameters ---
# These parameters mirror the PCF conditions you provided
BULLISH_PERCENT_MOVE = 1.08  # 8% increase (c/c5 >= 1.08)
BULLISH_DOLLAR_MOVE = 5.0    # $5 increase (c - c5 > 5)

BEARISH_PERCENT_MOVE = 0.92  # 8% decrease (c/c5 < 0.92)
BEARISH_DOLLAR_MOVE = 5.0    # $5 decrease (c5 - c > 5)

MIN_AVG_VOLUME_3_DAYS = 100000 # minv3.1 > 100000
MIN_PRICE_FILTER = 5.0       # c >= 5

# --- Data Fetching (Using yfinance) ---
# Removed the get_historical_data function as yf.download will be used directly with threading

# --- Scan Logic Functions ---

def meets_common_filters(df):
    """
    Checks common filters: minimum volume and minimum price.
    Assumes df is sorted by date with the most recent date at the end.
    """
    if df.empty or len(df) < 3: # Need at least 3 days for 3-day avg volume
        return False

    current_close = df['Close'].iloc[-1].item()
    # Calculate 3-day average volume for the most recent 3 days
    # This corresponds to 'minv3.1' in TeleChart PCF
    avg_volume_3_days = (df['Volume'].iloc[-3:].sum() / 3.0).item()

    volume_condition_met = avg_volume_3_days > MIN_AVG_VOLUME_3_DAYS
    price_condition_met = current_close >= MIN_PRICE_FILTER


    return volume_condition_met and price_condition_met

def scan_bullish_momentum_burst(df):
    """
    Scans for bullish momentum burst conditions:
    c/c5 >= 1.08 OR c - c5 > 5
    AND minv3.1 > 100000 AND c >= 5
    """
    if df.empty or len(df) < 6: # Need at least 6 days of data for c and c5 (index -1 and -6)
        return False

    current_close = df['Close'].iloc[-1].item()
    close_5_days_ago = df['Close'].iloc[-6].item() # -6 for 5 days prior to current_close

    # Check the percentage move
    percent_move_condition = (current_close / close_5_days_ago) >= BULLISH_PERCENT_MOVE

    # Check the dollar move
    dollar_move_condition = (current_close - close_5_days_ago) > BULLISH_DOLLAR_MOVE

    # Combine the price movement conditions and evaluate if any are true
    price_move_condition = percent_move_condition or dollar_move_condition

    # Check common filters (volume and current price)
    common_filters_met = meets_common_filters(df)

    return price_move_condition and common_filters_met

def scan_bearish_momentum_burst(df):
    """
    Scans for bearish momentum burst conditions:
    c/c5 < 0.92 OR c5 - c > 5
    AND minv3.1 > 100000 AND c >= 5
    """
    if df.empty or len(df) < 6: # Need at least 6 days of data for c and c5 (index -1 and -6)
        return False

    current_close = df['Close'].iloc[-1].item()
    close_5_days_ago = df['Close'].iloc[-6].item() # -6 for 5 days prior to current_close

    # Check the percentage move
    percent_move_condition = (current_close / close_5_days_ago) < BEARISH_PERCENT_MOVE

    # Check the dollar move
    dollar_move_condition = (close_5_days_ago - current_close) > BEARISH_DOLLAR_MOVE

    # Combine the price movement conditions and evaluate if any are true
    price_move_condition = percent_move_condition or dollar_move_condition

    # Check common filters (volume and current price)
    common_filters_met = meets_common_filters(df)

    return price_move_condition and common_filters_met

# --- Calculate Returns ---

def calculate_returns(df, period_start_date, period_end_date):
    """
    Calculates the percentage return for a given period.
    Assumes df is sorted by date. Finds the nearest available trading day for the start date.
    """
    # Find the index of the first trading day on or after the period_start_date
    start_idx = df.index.searchsorted(period_start_date, side='left')

    # If start_idx is out of bounds or the date at start_idx is before period_start_date,
    # find the next available trading day.
    if start_idx >= len(df) or df.index[start_idx] < period_start_date:
         # If the last date in the index is before the start date, return None
         if df.index[-1] < period_start_date:
             return None
         # Otherwise, find the first date in the index that is on or after period_start_date
         valid_start_dates = df.index[df.index >= period_start_date]
         if valid_start_dates.empty:
             return None
         start_date_actual = valid_start_dates[0]
    else:
        start_date_actual = df.index[start_idx]


    # Find the index of the last trading day on or before the period_end_date
    end_idx = df.index.searchsorted(period_end_date, side='right') - 1

    # If end_idx is out of bounds or the date at end_idx is after period_end_date,
    # find the previous available trading day.
    if end_idx < 0 or df.index[end_idx] > period_end_date:
        # If the first date in the index is after the end date, return None
        if df.index[0] > period_end_date:
            return None
        # Otherwise, find the last date in the index that is on or before period_end_date
        valid_end_dates = df.index[df.index <= period_end_date]
        if valid_end_dates.empty:
            return None
        end_date_actual = valid_end_dates[-1]
    else:
        end_date_actual = df.index[end_idx]


    # Ensure start_date_actual is not after end_date_actual
    if start_date_actual is None or end_date_actual is None or start_date_actual > end_date_actual:
        return None


    start_price = df.loc[start_date_actual, 'Close']
    end_price = df.loc[end_date_actual, 'Close']


    if start_price is not None and end_price is not None and start_price != 0:
        return ((end_price - start_price) / start_price) * 100
    else:
        return None


# --- Main Scanning Process ---
if __name__ == "__main__":
    # Example list of Nasdaq-listed stocks (replace with your actual list)
    # In a real scenario, you'd fetch a list of all Nasdaq tickers.
    # A simple way to get a large list for demonstration:
    # This is not a perfect list of all Nasdaq tickers, but provides a larger sample.
    try:
        # Read tickers from nasdaq.csv
        nasdaq_df = pd.read_csv('nasdaq.csv')
        # Assuming the column with tickers is named 'Symbol'
        if 'Symbol' in nasdaq_df.columns:
            # Convert the 'Symbol' column to string type and drop any NaN values
            nasdaq_tickers = nasdaq_df['Symbol'].astype(str).dropna().tolist()
        elif 'symbol' in nasdaq_df.columns: # Check for lowercase 'symbol' too
             # Convert the 'symbol' column to string type and drop any NaN values
             nasdaq_tickers = nasdaq_df['symbol'].astype(str).dropna().tolist()
        else:
            print("Could not find 'Symbol' or 'symbol' column in nasdaq.csv. Using a small default list.")
            nasdaq_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'TSLA', 'META', 'NFLX', 'INTC', 'CSCO',
                      'QCOM', 'AMD', 'ADBE', 'PYPL', 'CMCSA', 'SBUX', 'BKNG', 'CHTR', 'GILD', 'MDLZ']

        # Replace '/' with '.' in ticker symbols
        nasdaq_tickers = [ticker.replace('/', '.') for ticker in nasdaq_tickers]

    except FileNotFoundError:
        print("Error: nasdaq.csv not found at /nasdaq.csv. Using a small default list for demonstration.")
        nasdaq_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'TSLA', 'META', 'NFLX', 'INTC', 'CSCO',
                      'QCOM', 'AMD', 'ADBE', 'PYPL', 'CMCSA', 'SBUX', 'BKNG', 'CHTR', 'GILD', 'MDLZ']
        # Replace '/' with '.' in default list as well
        nasdaq_tickers = [ticker.replace('/', '.') for ticker in nasdaq_tickers]
    except Exception as e:
        print(f"Could not read tickers from /nasdaq.csv: {e}")
        print("Using a small default list.")
        nasdaq_tickers = ['AAPL', 'MSFT', 'GOOGL', 'AMZN', 'NVDA', 'TSLA', 'META', 'NFLX', 'INTC', 'CSCO',
                      'QCOM', 'AMD', 'ADBE', 'PYPL', 'CMCSA', 'SBUX', 'BKNG', 'CHTR', 'GILD', 'MDLZ']
        # Replace '/' with '.' in default list as well
        nasdaq_tickers = [ticker.replace('/', '.') for ticker in nasdaq_tickers]


    bullish_candidates = []
    bearish_candidates = []
    returns_data = []

    print("--- Starting Momentum Burst Scan and Returns Calculation ---")
    print(f"Scanning {len(nasdaq_tickers)} stocks for 5-day explosive moves and calculating returns using yfinance with 20 threads...")
    print("-" * 40)

    # Define the date range for historical data (sufficient for 5-day lookback + volume and 1-year return)
    end_date = datetime.now()
    start_date = end_date - timedelta(days=400) # Fetching enough days for the scan and 1-year return

    # Fetch data for all tickers using yf.download with threading
    # yf.download returns a pandas DataFrame with a MultiIndex (columns: 'Adj Close', 'Close', etc., index: 'Date', 'Ticker')
    all_stock_data = yf.download(nasdaq_tickers, start=start_date, end=end_date, threads=20)

    # Check if data was downloaded successfully
    if all_stock_data.empty:
        print("Could not download data for any tickers. Please check the ticker list and yfinance connectivity.")
    else:
        # yf.download can return a DataFrame with a single level column index if only one ticker is downloaded.
        # Ensure a consistent MultiIndex for easier processing.
        if isinstance(all_stock_data.columns, pd.MultiIndex):
             # Transpose and swap levels to get Ticker as the primary index level
             all_stock_data = all_stock_data.stack(level=0).unstack(level=1)
        else:
             # If only one ticker, add a ticker level to the columns
             all_stock_data.columns = pd.MultiIndex.from_product([all_stock_data.columns, [nasdaq_tickers[0]]])
             all_stock_data = all_stock_data.stack(level=0).unstack(level=1)


        # Determine the latest date in the downloaded data
        latest_date = all_stock_data.index.max()

        # Define the start dates for different periods and convert them to datetime64[ns]
        one_year_ago = latest_date - timedelta(days=365)
        ytd_start = pd.to_datetime(datetime(latest_date.year, 1, 1).date())
        qtd_start = pd.to_datetime(datetime(latest_date.year, (latest_date.month - 1) // 3 * 3 + 1, 1).date())
        mtd_start = pd.to_datetime(datetime(latest_date.year, latest_date.month, 1).date())
        # For WTD, find the most recent Monday and convert to datetime64[ns]
        wtd_start = pd.to_datetime(latest_date - timedelta(days=latest_date.weekday()))


        # Now iterate through each ticker's data in the downloaded DataFrame
        for ticker in nasdaq_tickers:
            # Get the historical data for the current ticker
            # yf.download might include tickers that failed to download, resulting in a column of NaNs
            if ticker in all_stock_data.columns.get_level_values(0):
                 stock_data = all_stock_data[ticker].copy() # Explicitly create a copy
                 # Drop rows with NaN values that might occur if data is incomplete for a ticker
                 stock_data.dropna(inplace=True)

                 if not stock_data.empty and len(stock_data) >= 6: # Ensure enough data points after dropping NaNs for scans
                    # Step 2: Apply the bullish scan
                    if scan_bullish_momentum_burst(stock_data):
                        bullish_candidates.append(ticker)

                    # Step 3: Apply the bearish scan
                    if scan_bearish_momentum_burst(stock_data):
                        bearish_candidates.append(ticker)

                    # Calculate returns for each period
                    ytd_return = calculate_returns(stock_data, ytd_start, latest_date)
                    qtd_return = calculate_returns(stock_data, qtd_start, latest_date)
                    mtd_return = calculate_returns(stock_data, mtd_start, latest_date)
                    wtd_return = calculate_returns(stock_data, wtd_start, latest_date)
                    one_year_return = calculate_returns(stock_data, one_year_ago, latest_date)

                    returns_data.append({
                        'Ticker': ticker,
                        'WTD Return (%)': wtd_return,
                        'MTD Return (%)': mtd_return,
                        'QTD Return (%)': qtd_return,
                        'YTD Return (%)': ytd_return,
                        '1Y Return (%)': one_year_return
                    })

                 # else:
                     # print(f"Skipping {ticker}: Not enough valid data points after dropping NaNs.")
            # else:
                # print(f"Skipping {ticker}: Data not found in downloaded data.")


    print("\n--- Scan Results ---")
    if bullish_candidates:
        print("\nBullish Momentum Burst Candidates (8%+ or $5+ up in 5 days):")
        for ticker in bullish_candidates:
            print(f"- {ticker}")
    else:
        print("\nNo bullish momentum burst candidates found today.")

    if bearish_candidates:
        print("\nBearish Momentum Burst Candidates (8%+ or $5+ down in 5 days):")
        for ticker in bearish_candidates:
            print(f"- {ticker}")
    else:
        print("\nNo bearish momentum burst candidates found today.")

    # Create a DataFrame to store the returns
    returns_df = pd.DataFrame(returns_data)

    # --- Save to Excel ---
    excel_filename = 'nasdaq_returns.xlsx'
    returns_df.to_excel(excel_filename, index=False)

    print(f"\n--- Returns Calculation Complete ---")
    print(f"Returns saved to {excel_filename}")

    print("\n--- Scan and Returns Calculation Complete ---")
    print("Note: This script now uses real data from yfinance with threading, but results depend on data availability and scan parameters.")

--- Starting Momentum Burst Scan and Returns Calculation ---
Scanning 6943 stocks for 5-day explosive moves and calculating returns using yfinance with 20 threads...
----------------------------------------


[*********************100%***********************]  6942 of 6942 completed
ERROR:yfinance:
399 Failed downloads:
ERROR:yfinance:['BHR^D', 'LFT^A', 'MS^E', 'SF^C', 'FLG^A', 'WFC^A', 'ATH^D', 'WFC^D', 'T^C', 'SCE^K', 'TFC^R', 'KIM^M', 'GS^C', 'AHT^D', 'REXR^B', 'SCE^M', 'ALL^B', 'CMRE^B', 'EFC^D', 'CMS^B', 'GAM^B', 'AGM^E', 'SAND          ', 'MFA^B', 'ARGO^A', 'KEY^K', 'SPG^J', 'GPMT^A', 'GSL^B', 'SEAL^B', 'HEI.A', 'RWT^A', 'USB^S', 'HPP^C', 'BML^G', 'USB^Q', 'MS^P', 'KEY^L', 'BAC^E', 'BC^A', 'F^C', 'BIP^A', 'PCG^H', 'NEE^S', 'SLG^I', 'EFC^B', 'BFS^E', 'PRIF^K', 'BCV^A', 'ICR^A', 'RLJ^A', 'PRIF^D', 'REXR^C', 'ALB^A', 'MS^A', 'ALL^I', 'GUT^C', 'CHMI^B', 'HIG^G', 'GAB^K', 'TRTN^C', 'SB^C', 'OPP^A', 'RITM^D', 'FLG^U', 'ACP^A', 'NEE^R', 'KEY^I', 'CTO^A', 'CLDT^A', 'RF^C', 'GGT^E', 'AKO.B', 'SF^D', 'PSA^I', 'CIO^A', 'ANG^B', 'ATCO^D', 'BA^A', 'ACR^C', 'CIM^C', 'ASB^E', 'PBI^B', 'ATH^B', 'COF^I', 'AHL^E', 'WRB^F', 'PCG^X', 'RC^E', 'GLP^B', 'PMT^B', 'GGN^B', 'VNO^M', 'AHT^G', 'JPM^D', 'FRT^C', 


--- Scan Results ---

Bullish Momentum Burst Candidates (8%+ or $5+ up in 5 days):
- AAL
- AAP
- ABG
- ACHR
- ADUR
- AEIS
- AES
- AGIO
- AIT
- ALAB
- ALB
- ALGN
- ALGS
- ALLT
- ALMU
- ALTG
- AMAT
- AMD
- AMG
- AMLX
- AMPX
- AMRC
- ANAB
- ANET
- ANSS
- AON
- APDN
- APLS
- APO
- APP
- APPF
- ARGX
- ARQT
- ARRY
- ARTL
- ARWR
- ASO
- ASPN
- ASST
- ATAT
- ATXS
- AUPH
- AVDL
- AVGO
- AXGN
- AXSM
- AZ
- AZO
- AZZ
- BA
- BBIO
- BCYC
- BE
- BEAM
- BEKE
- BIO
- BKD
- BKKT
- BKNG
- BKSY
- BLD
- BLDR
- BLK
- BNTC
- BNTX
- BOOT
- BRSL
- BSET
- BTCM
- BUR
- BURL
- BX
- CADL
- CAPR
- CAR
- CASY
- CAT
- CAVA
- CC
- CDNS
- CEG
- CEP
- CIFR
- CLF
- CLRO
- CMI
- CNVS
- COGT
- COHR
- COIN
- COLL
- COR
- CPRI
- CRCL
- CRDO
- CREV
- CRL
- CRNX
- CRSP
- CRVO
- CSIQ
- CSL
- CSW
- CTEV
- CVI
- CVX
- DAL
- DALN
- DFDV
- DK
- DKS
- DLTR
- DNTH
- DPZ
- DQ
- DXYZ
- ECOR
- EG
- EGG
- ELVN
- EME
- ENVX
- EPSM
- ERIE
- ESLT
- ETSY
- EVOK
- EVR
- EXP
- EXPE
- EXPI
- EYPT
- FGEN
- FINV
- FLNC
- FLUT
- FMFC
- FOUR
- FR